In [1]:
!ls

 Anaconda_vs_Miniconda
'Approaching categorical vars.ipynb'
 bin
 catinthedatii
 C_Indrani_Aditya_Odyssey_Dharani.pdf
 C_Indrani_Harrys_Mansion_Dharani.pdf
'Cross validation .ipynb'
 Desktop
 Documents
 Downloads
 environment.yml
 examples.desktop
 miniconda3
 mnist_dataset.ipynb
 mnist_train.csv
 Music
'Overfitting example using winequality-red data.ipynb'
 Pictures
 project
 Public
 requirements.txt
 scikit_learn_data
 snap
 Templates
 Untitled.ipynb
 untitled.txt
 Videos
 winequality-red.csv
 winequality-red.ipynb


In [2]:
mapping = {
    "Freezing": 0,
    "Warm": 1,
    "Cold": 2,
    "Boiling Hot": 3,
    "Hot": 4,
    "Lava Hot": 5
}

import pandas as pd

df = pd.read_csv("/home/anand/catinthedatii/cat_train.csv")

df.loc[:,"ord_2"] = df.ord_2.map(mapping)

# Label Encoding i.e we are encoding every category as a numerical label

In [3]:
df.ord_2.value_counts()

0.0    142726
1.0    124239
2.0     97822
3.0     84790
4.0     67508
5.0     64840
Name: ord_2, dtype: int64

In [6]:
import pandas as pd
from sklearn import preprocessing

# read the data
df = pd.read_csv("/home/anand/catinthedatii/cat_train.csv")

# fill NaN values in ird_2 column
df.loc[:,"ord_2"] = df.ord_2.fillna("NONE")

# initialize LabelEncoder
lbl_enc = preprocessing.LabelEncoder()

# fit label encoder and transform values on ord_2 column
# P.S - do not use this directly. fit first, then transform.
df.loc[:,"ord_2"] = lbl_enc.fit_transform(df.ord_2.values)



## We can use this directly in many tree-based models:

Decision Trees,
Random Forests,
Extra Trees,
Or any kind of boosted trees moodel -
    XGBoost,
    GBM,
    LightGBM.


This type of encoding cannot be used in linear models, support vector machines or neural networks as they expect data to be normalized (or standardized).
For these type of models, we can binarize the data.

Freezing --> 0 --> 0 0 0
Warm     --> 1 --> 0 0 1
Cold     --> 2 --> 0 1 0
Boiling Hot >3 --> 0 1 1
Hot      --> 4 --> 1 0 0
Lava Hot --> 5 --> 1 0 1


In [6]:
# Introducing sparse format for binary variables

import numpy as np

# create our example of feature matrix
example = np.array(
    [
        [0,0,1],
        [1,0,0],
        [1,0,1]
    ]
)

#PRINT SIZE IN BYTES
print(example.nbytes)

72


In [8]:
import numpy as np
from scipy import sparse

# One way to represent this matrix only with 1s would be some kind of dictionary method in which 
#keys are indices of rows and columns and value is 1

# convert numpy array to sparse csr matrix
sparse_example = sparse.csr_matrix(example)

#print size of this sparse matrix
print(sparse_example.data.nbytes)

32


In [13]:
print(sparse_example)

  (0, 2)	1
  (1, 0)	1
  (2, 0)	1
  (2, 2)	1


In [14]:
print(sparse_example.data.nbytes +
     sparse_example.indptr.nbytes +
     sparse_example.indices.nbytes)

64


In [17]:
import numpy as np
from scipy import sparse

n_rows = 1000
n_cols = 100000

# create a random binary matrix with only 5% values as 1s
example = np.random.binomial(1,p=0.05,size=(n_rows,n_cols))

# print size in bytes
print(f"Size of dense array: {example.nbytes}")

# convert numpy array to sparse CSR matrix
sparse_example = sparse.csr_matrix(example)

# print size of this in sparse matrix
print(f"Size of sparse array: {sparse_example.data.nbytes}")

full_size = (
    sparse_example.data.nbytes +
    sparse_example.indptr.nbytes +
    sparse_example.indices.nbytes
)

print(f"Full size of the sparse array: {full_size}")


Size of dense array: 800000000
Size of sparse array: 39974344
Full size of the sparse array: 59965520


### Check other ways of representing a sparse matrix. Above mentioned is one way and a popular way

# One hot encoding

Vector having only 1s and 0s and of the size = no. of categories in the variable

In [ ]:
import numpy as np
from sklearn import preprocessing

# create random 1-d array with 1001 different categories (int)
exanple = np.random.randint(1000,size=1000000)

# Initialize one hot encoder from scikit-learn
# keep sparse = False to get dense array
ohe = preprocessing.OneHotEncoder(sparse=False)

# fit and transform data with dense one hot encoder
ohe_example = ohe.fit_transform(example.reshape(-1,1))

# print size in bytes for dense array
print(f"Size of dense array: {ohe_example.data.nbytes}")

# Initialise OneHotEncoder from scikit-learn
# keep sparse = True to get sparse array
ohe = preprocessing.OneHotEncoder(sparse=True)

# fit and transform data with sparse one hot encoder
ohe_example = ohe.fit_transform(example.reshape(-1,1))

# print size of this sparse matrix
print(f"Size of sparse array: {ohe_example.data.nbytes}")

full_size = (
    ohe_example.data.nbytes +
    ohe_example.indptr.nbytes +
    ohe_example.indices.nbytes
)

# print full size of this sparse matrix
print(f"Full size of sparse array: {full_size}")

# Handling missing values and rare values

In [4]:
import pandas as pd
from sklearn import preprocessing

# read training data
train = pd.read_csv("/home/anand/catinthedatii/cat_train.csv")

# read test data
test = pd.read_csv("/home/anand/catinthedatii/cat_test.csv")

# create a fake target column for the test data so that we can distinguish test/train later
test.loc[:,"target"] = -1

# concatenate test and train data
data = pd.concat([train,test]).reset_index(drop=True)

# make a list of features we are interested in
# id and target is something we should not encode

features = [x for x in train.columns if x not in ["id","target"]]

# loop over the features list
for feat in features:
    # create a new instance of Labelencoder for each feature
    lbl_enc = preprocessing.LabelEncoder()
    
    # note this trick here
    # since its categorical data, we fillna with a string
    # and we convert all data to string type so no matter its int or float, its converted to string
    temp_col = data[feat].fillna("NONE").astype(str).values
    
    # we can use fit_transform here as we do not have any extra test data that we need to transform seperately
    data.loc[:,feat] = lbl_enc.fit_transform(temp_col)
    

# split the train and test dataset again
train = data[data.target != -1].reset_index(drop=True)
test = data[data.target == -1].reset_index(drop=True)

# this trick wont work in a live setting if a new category comes up as in example of Real time bidding (RTB)


## Assume that the test data will have the same categories as training or you can intriduce a rare or unknown category to training to take care of new categories in test data

In [7]:
df.ord_4.value_counts()

N    39978
P    37890
Y    36657
A    36633
R    33045
U    32897
M    32504
X    32347
C    32112
H    31189
Q    30145
T    29723
O    25610
B    25212
E    21871
K    21676
I    19805
D    17284
F    16721
W     8268
Z     5790
S     4595
G     3404
V     3107
J     1950
L     1657
Name: ord_4, dtype: int64

In [9]:
df.ord_4 = df.ord_4.fillna("NONE")
df.ord_4.value_counts()

N       39978
P       37890
Y       36657
A       36633
R       33045
U       32897
M       32504
X       32347
C       32112
H       31189
Q       30145
T       29723
O       25610
B       25212
E       21871
K       21676
I       19805
NONE    17930
D       17284
F       16721
W        8268
Z        5790
S        4595
G        3404
V        3107
J        1950
L        1657
Name: ord_4, dtype: int64

In [11]:
df.loc[df.ord_4.value_counts()[df["ord_4"]].values < 2000,
      "ord_4"
      ] = "RARE"

df.ord_4.value_counts()

N       39978
P       37890
Y       36657
A       36633
R       33045
U       32897
M       32504
X       32347
C       32112
H       31189
Q       30145
T       29723
O       25610
B       25212
E       21871
K       21676
I       19805
NONE    17930
D       17284
F       16721
W        8268
Z        5790
S        4595
RARE     3607
G        3404
V        3107
Name: ord_4, dtype: int64

In [12]:
df.ord_4.value_counts()[df["ord_4"]].values < 2000

array([False, False, False, ..., False, False, False])

In [14]:
df.target.value_counts()

0    487677
1    112323
Name: target, dtype: int64